# GUI Implementierung (othello_gui.ipynb)
\label{sec:gui}
\ifx false

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

\fi Im folgenden Abschnitt wird eine \ac{GUI} für das Spiel Othello implementiert. Diese ermöglicht das Anzeigen Spielfelds und das Setzen von Steinen durch einen menschlichen Spieler per Mausklick. Außerdem wird ein Einstellungsmenü implementiert, über welches das Spiel und die \ac{KI}-Agenten konfiguriert werden können.

## Importieren der externen Abhängigkeiten

Die \ac{GUI} verwendet zur Darstellung des Spielzustandes, zum Anzeigen erweiterter Informationen sowie für die Benutzerinteraktion die Bibliotheken `ipycanvas` und `ipywidgets`. Diese können direkt im Jupyter Notebook verwendet werden.

Zusätzlich werden aus dem Paket `math` der Python Standardbibliothek die Variable `pi` sowie die Funktion `floor` benötigt.

In [ ]:
import ipycanvas
import ipywidgets
import math
from ipywidgets import RadioButtons, HBox, VBox, IntSlider, Label

## Konfiguration der GUI

Die \ac{GUI} unterstützt optional die Anzeige aller möglichen Züge in der aktuellen Spielsituation, sowie die Darstellung der Menge `frontier`, welche im \autoref{sec:gamelogic} beschrieben wird. Ob diese Features verwendet werden wird im Folgenden mit den Konstanten `SHOW_FRONTIER` und `SHOW_POSSIBLE_MOVES` konfiguriert.

In [ ]:
SHOW_FRONTIER = False
SHOW_POSSIBLE_MOVES = True

## Canvas Initialisierung

Zunächst wird ein Canvas Objekt der Bibliothek `ipycanvas` initialisiert, auf dem später das Spielfeld dargestellt wird. Die Konstante `CELL_SIZE` beschreibt hierbei die Größe eines einzelen Feldes in Pixeln auf dem Spielbrett, wohingegen `CANVAS_SIZE` die Größe des gesamten Spielbretts angibt.

In [ ]:
CELL_SIZE = 60

CANVAS_SIZE = BOARD_SIZE * CELL_SIZE

canvas = ipycanvas.MultiCanvas(2, width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas[0].fill_style = 'darkgreen'
canvas[0].stroke_style = 'black'
canvas[0].fill_rect(0, 0, CANVAS_SIZE, CANVAS_SIZE)
canvas[0].begin_path()
for i in range(BOARD_SIZE+1):
    pos = i * CELL_SIZE
    canvas[0].move_to(pos, 0)
    canvas[0].line_to(pos, CANVAS_SIZE)
    canvas[0].move_to(0, pos)
    canvas[0].line_to(CANVAS_SIZE, pos)
canvas[0].stroke()

## Widget Initialisierung

Unter dem Spielfeld werden mit der Bibliothek `ipywidgets` zusätzliche Informationen zum aktuellen Spielzustand angezeigt. Diese werden mithilfe sogenannter Widgets dargestellt, welche im Folgenden erzeugt werden.

Das `score_lbl` Widget enthält die Steinzahl beider Spieler im aktuellen Spielzustand.

In [ ]:
score_lbl = ipywidgets.widgets.Label()

Das `turn_lbl` Widget nennt den Spieler, der gerade am Zug ist.

In [ ]:
turn_lbl = ipywidgets.widgets.Label()

Das `output` Widget macht die Ausgabe mithilfe von `print()` sowie die Ausgabe von Fehlermeldungen trotz der Verwendung von IPyWidgets und IPyCanvas möglich.

In [ ]:
output = ipywidgets.widgets.Output()

Das Widget `utility_lbl` dient der Anzeige der von den \ac{KI} Spielern geschätzten Nützlichkeit des Spielzustands.

In [ ]:
utility_lbl = ipywidgets.widgets.Label()

In der Funktion `update_output` wird der Spielzustand `state` auf den existierenden Canvas gezeichnet.

In [ ]:
def update_output(state):
    with ipycanvas.hold_canvas(canvas):
        canvas[1].clear()
        for ((x, y), val) in np.ndenumerate(state.board):
            if val == NONE:
                continue
            elif val == BLACK:
                canvas[1].fill_style = 'black'
            else:
                canvas[1].fill_style = 'white'
            canvas[1].fill_arc((x+0.5) * CELL_SIZE, (y+0.5)
                               * CELL_SIZE, CELL_SIZE/2.2, 0, 2*math.pi)

        if state.last_move != None:
            (x, y) = state.last_move
            canvas[1].stroke_style = 'red'
            canvas[1].line_width = 2
            canvas[1].stroke_arc((x+0.5) * CELL_SIZE, (y+0.5)
                               * CELL_SIZE, CELL_SIZE/2.2, 0, 2*math.pi)

        if SHOW_FRONTIER:
            for (x, y) in state.frontier:
                canvas[1].fill_style = 'gray'
                canvas[1].fill_arc((x+0.5) * CELL_SIZE, (y+0.5)
                                   * CELL_SIZE, CELL_SIZE/6, 0, 2*math.pi)

        if SHOW_POSSIBLE_MOVES:
            for (x, y) in get_possible_moves(state, state.turn):
                if state.turn == BLACK:
                    canvas[1].fill_style = 'black'
                else:
                    canvas[1].fill_style = 'white'
                canvas[1].fill_arc((x+0.5) * CELL_SIZE, (y+0.5)
                                   * CELL_SIZE, CELL_SIZE/6, 0, 2*math.pi)

    b_score = count_disks(state, BLACK)
    w_score = count_disks(state, WHITE)
    score_lbl.value = f'Black Player : {b_score} White Player : {w_score}'
    b_util = utilities[BLACK]
    w_util = utilities[WHITE]
    utility_lbl.value = f'Utility: Black: {b_util} / White: {w_util}'
    if state.game_over:
        turn_lbl.value = f'{get_player_string(get_utility(state))} wins'
    else:
        turn_lbl.value = f'{get_player_string(state.turn)}s Move'

Die Funktion `display_board` stellt den durch `state` angegebenen Spielzustand dar, indem zunächst der Canvas per `update_output` aktualisiert, und dann zusammen mit den Status-Widgets angezeigt wird.

In [ ]:
def display_board(state):
    output.clear_output()
    update_output(state)
    display(canvas)
    display(score_lbl)
    display(turn_lbl)
    display(utility_lbl)
    display(output)

Für menschliche Spieler muss festgestellt werden, ob auf das Spielfeld geklickt wurde. Dies geschieht in der Callback Funktion `mouse_down`, welche die x und y Koordinate des Mausklicks relativ zum Canvas erhält. Auf Basis dieser Position wird, falls möglich, ein Zug auf das angeklickte Feld gemacht. Die Funktion wird durch den Aufruf von `on_mouse_down` auf dem IPyCanvas als Callback Funktion registriert.

In [ ]:
def mouse_down(x_px, y_px):
    global state
    with output:
        if not state.game_over:
            x = math.floor(x_px / CELL_SIZE)
            y = math.floor(y_px / CELL_SIZE)
            try:
                state = make_move(state, (x, y))
            except InvalidMoveException:
                print('Invalid Move')
            update_output(state)
            try:
                next_move(state)
            except KeyboardInterrupt:
                pass

canvas[1].on_mouse_down(mouse_down)

## Spieleinstellungen

Hier wird durch Nutzung von `ipywidgets` eine \ac{GUI} erzeugt, über die für beide Spieler Einstellungen vorgenommen werden können. So kann zum Beispiel festelegt werden, ob ein Spieler vom Nutzer, oder von einem \ac{KI}-Agenten kontrolliert werden soll und Parameter der \ac{KI} angepasst werden.

In [ ]:
algorithms = { 'Menschlicher Spieler': None,
               'ProbCut': probcut,
               'Alpha-Beta': alphabeta,
               'Minimax': minimax,
               'Zufällig': random_ai }
modes = { 'Feste Tiefe': ai_make_move,
          'Iterative Vertiefung': ai_make_move_id,
          'Zeitbegrenzte Vertiefung': ai_make_move_id_timelimited }
heuristics = { 'Cowthello': cowthello_heuristic,
               'Mobilität': mobility_heuristic,
               'Kombiniert': combined_heuristic }
black_algorithm = RadioButtons(
    options=algorithms.keys(),
    value='Menschlicher Spieler',
    description='Schwarz:'
)
black_heuristic = RadioButtons(
    options=heuristics.keys(),
    value='Kombiniert'
)
black_mode = RadioButtons(
    options=modes.keys(),
    value='Zeitbegrenzte Vertiefung'
)
black_depth = IntSlider(value=5, min=1, max=10,
                        description='Suchtiefe:')
black_timelimit = IntSlider(value=30, min=1, max=120,
                            description='Zeitlimit:')
black_ints = VBox([black_depth, black_timelimit])
black_config = HBox([black_algorithm, black_mode,
                     black_heuristic, black_ints])

white_algorithm = RadioButtons(
    options=algorithms.keys(),
    value='ProbCut',
    description='Weiß:'
)
white_heuristic = RadioButtons(
    options=heuristics.keys(),
    value='Kombiniert'
)
white_mode = RadioButtons(
    options=modes.keys(),
    value='Zeitbegrenzte Vertiefung'
)
white_depth = IntSlider(value=5, min=1, max=10,
                        description='Suchtiefe:')
white_timelimit = IntSlider(value=30, min=1, max=120,
                            description='Zeitlimit:')
white_ints = VBox([white_depth, white_timelimit])
white_config = HBox([white_algorithm, white_mode,
                     white_heuristic, white_ints])
settings = ipywidgets.VBox([black_config, white_config])

Die Funktion `configure_settings` dient der Anzeige des Konfigurationsmenüs.

In [ ]:
def configure_settings():
    display(settings)

Die Funktion `get_settings` liefert die über das Konfigurationsmenü getätigte Konfiguration als Dictionary zurück. Dies wird später im Frontend verwendet.

In [ ]:
def get_settings():
    return { BLACK: { 'heuristic': heuristics[black_heuristic.value],
                      'algorithm': algorithms[black_algorithm.value],
                      'depth': black_depth.value,
                      'timelimit': black_timelimit.value,
                      'mode': modes[black_mode.value] },
             WHITE: { 'heuristic': heuristics[white_heuristic.value],
                      'algorithm': algorithms[white_algorithm.value],
                      'depth': white_depth.value,
                      'timelimit': white_timelimit.value,
                      'mode': modes[white_mode.value] }}